#Explicação do Projeto
O objetivo do projeto é criar um corretor ortográfico que, dada a escrita incorreta de uma palavra, sugira uma palavra correta que se mais se aproxime.

#Pré condição:
A base de dados ultilizada, artigo, foi tratada para que as imagens contidas no artigo fossem substituídas pela palavra 'imagem', assim como os códigos contidos fossem substituídos pela linguagem em que foram excritos, por exeplo: java, python, etc.

#Preparação de dados

In [ ]:
#imports
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [ ]:
with open("artigos.txt", "r") as f:
  artigos = f.read()

print(artigos[:100])




imagem 

Temos a seguinte classe que representa um usuário no nosso sistema:

java

Para salvar u


In [ ]:
tokens = nltk.tokenize.word_tokenize(artigos)

Essa variável tokens está carregando além das palavras da base de dados, todas as pontuações, então será feito uma filtragem apenas com tokens alfa-numéricos, afim de ficar apenas com as palavras.

In [25]:
def separa_e_normaliza_palavras(lista_tokens):
    lista_palavras = []
    for token in lista_tokens:
      if token.isalpha():
        lista_palavras.append(token.lower())
    return lista_palavras

palavras_separadas = separa_e_normaliza_palavras(tokens)
print(len(set(palavras_separadas)))

18465


#Primeiro corretor: Esquece uma letra.

A primeira funcionalidade implementada ao corretor é com o objetivo de corrigir esquecimentos de letras, por exemplo na digitação de "comptador" ao invés de "computador". Para isso a palavra é divida em 2 partes para cada caso possível dentro da palavra, ou seja:

('', 'comptador'), ('c', 'omptador'), ('co', 'mptador'), etc.

Tendo essas fatias consideradas partes Esquerda e Direita da palavra, será concatenado essas partes com cada letra do alfabeto, gerando uma lista de palavras sugeridas.
Por fim, o corretor irá avaliar qual das palavras sugeridas possui, dentro da base de dados usada, a maior frequência de aparição, sugerindo então que a palavra desejada seja esta.

In [ ]:
def insere_letras(fatias):
  palavras_sugeridas = []
  letras = 'abcdeifghijklmnopqrstuvwxyzáãàâéèêíìîóõòôúùûç-'
  for E, D in fatias:
    for letra in letras:
      palavras_sugeridas.append(E + letra + D)
  return palavras_sugeridas

In [ ]:
def gerador_de_palavras(palavra):
  fatias = []

  for i in range(len(palavra)+1):
    fatias.append((palavra[:i],palavra[i:]))
    
  palavras_geradas = insere_letras(fatias)
  return palavras_geradas

In [ ]:
frequencia = nltk.FreqDist(palavras_separadas)
total_de_palavras = len(palavras_separadas)

def probabilidade(palavra_gerada):
  return frequencia[palavra_gerada]/total_de_palavras

In [ ]:
def corretor(palavra):
  palavras_geradas = gerador_de_palavras(palavra)
  palavra_sugerida = max(palavras_geradas, key=probabilidade)
  return palavra_sugerida

#Implementação de um avaliador para as correções.

In [ ]:
def prepara_dados_teste():
  lista_palavras_teste = []
  f = open("palavras.txt", 'r')

  for linha in f:
    correta, errada = linha.split()
    lista_palavras_teste.append((correta, errada))
  f.close()
  return lista_palavras_teste

def avalia_corretor(dados_teste, vocabulario):
  numero_de_palavras = len(dados_teste)
  acertos = 0
  palavras_desconhecida = 0

  for correta, errada in dados_teste:
    palavras_desconhecida += (correta not in vocabulario)
    if (corretor(errada) == correta):
      acertos += 1      

  taxa_de_acerto = round(100*acertos/numero_de_palavras, 2)
  taxa_de_desconhecidas = round(palavras_desconhecida*100/numero_de_palavras, 2)
  print("A taxa de acerto foi de: " + f"{taxa_de_acerto}" + "% e a de palavras desconhecidas foi de: "
  + f"{taxa_de_desconhecidas}" + "%")

In [ ]:
avalia_corretor(prepara_dados_teste(), set(palavras_separadas))

A taxa de acerto foi de: 1.08% e a de palavras desconhecidas foi de: 6.99%


Foi implementado então uma função para avaliar a precisão (corretas / total) dos palpites do corretor. Além disso a função retorna quanto das palavras testadas não são conhecidas pela base de testes utilizada.

Pelo resultado obtido é possivel perceber que apenas uma pequena parte das palavras foi corrigida com a utilização apenas deste método.

#Segundo corretor: Deletando caracteres

A segunda funcionalidade é a de deletar letras digitadas além da palavra desejada, 'compuitador', por exemplo. Para isso, novamente é feito o fatiamento da palavra e deletada a primeira letra da parte da direita.

In [ ]:
def deleta_caracteres(fatias):
  palavras_sugeridas = []
  for E, D in fatias:
      palavras_sugeridas.append(E + D[1:])
  return palavras_sugeridas

In [ ]:
def gerador_de_palavras(palavra):
  fatias = []

  for i in range(len(palavra)+1):
    fatias.append((palavra[:i],palavra[i:]))
    
  palavras_geradas = insere_letras(fatias)
  palavras_geradas += deleta_caracteres(fatias)
  return palavras_geradas

In [ ]:
avalia_corretor(prepara_dados_teste(), set(palavras_separadas))

A taxa de acerto foi de: 41.4% e a de palavras desconhecidas foi de: 6.99%


Pode ser observada uma melhora na precisão, de 1% para 41%.

#Terceiro Corretor: Trocando letras

Na terceira mudança é implementada uma nova funcionalidade que deleta uma letra e coloca uma nova no lugar, além de uma função que troca duas letras de lugar para cobrir casos como: primeira funcionalidade 'compitador' ou segunda 'copmutador'.

In [30]:
def troca_letra(fatias):
    novas_palavras = []
    letras = 'abcdefghijklmnopqrstuvwxyzáâàãéêèẽíîìĩóôõòúûùũç'
    for E, D in fatias:
        for letra in letras:
            novas_palavras.append(E + letra + D[1:])
    return novas_palavras

def inverte_letra(fatias):
    novas_palavras = []
    for E, D in fatias:
        if len(D) > 1:
            novas_palavras.append(E + D[1] + D[0] + D[2:])
    return novas_palavras

In [35]:
def gerador_de_palavras(palavra):
  fatias = []

  for i in range(len(palavra)+1):
    fatias.append((palavra[:i],palavra[i:]))
    
  palavras_geradas = insere_letras(fatias)
  palavras_geradas += deleta_caracteres(fatias)
  palavras_geradas += troca_letra(fatias)
  palavras_geradas += inverte_letra(fatias)

  return palavras_geradas

In [36]:
avalia_corretor(prepara_dados_teste(), set(palavras_separadas))

A taxa de acerto foi de: 76.34% e a de palavras desconhecidas foi de: 6.99%


O resultado final encontrado foi este terceiro, com uma precisão de 76.34%, na tentativa de alcançar um modelo melhor algumas modificações não mostradas no resultado final foram implementadas, como por exemplo a reinserção das palavras geradas no corretor (tentando alcançar palavras com 2 letras escritas a mais, ou a menos), mas geraram resultados piores, neste caso porque começa a se distanciar do caso original, e por mais que para alguns casos acerte, na maioria erra palavras que estava já acertando. Isso acontece pelo modo como estamos avaliando a possibilidade de ser aquela palavra ou não (maior frequência na base de dados que utilizamos). Talvez seja justamente neste ponto que precisaria ser feito uma mudança para atingir melhores resultados.